# Data Preparation

## Imports

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline 

from shapely.geometry import Point
import geopandas as gpd
from geopandas import GeoDataFrame

import sklearn
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.cluster import KMeans

In [ ]:
df = pd.read_csv('vehicles.csv')

## Data Exploration

In [ ]:
df.head()

In [ ]:
df.columns

In [ ]:
df.shape

In [ ]:
x = (df.groupby(['manufacturer'],as_index = False).count().sort_values('price', ascending = False))['manufacturer'].tolist()
y = (df.groupby(['manufacturer'],as_index = False).count().sort_values('price', ascending = False))['price'].tolist()
fig = plt.figure(figsize=(15,10))
plt.xticks(rotation = 90)
plt.bar(x, y, width=1)

In [ ]:
#Information about the categorical variables

print(df["paint_color"].unique())
print(df["condition"].unique())
print(df["type"].unique())
print(df["fuel"].unique())
print(df["cylinders"].unique())
print(df["title_status"].unique())
print(df["type"].unique())
print(df["drive"].unique())
print(df["transmission"].unique())

In [ ]:
df.isnull().sum().sort_values(ascending = False)

In [ ]:
#a lot of locations that don't make sense (i.e. on Antarctica or in the ocean) 
#User can drop a pin anywhere in the world => lat & long values are unreliable
#Use 'state' and/or 'region' instead of lat & long for location

geometry = [Point(xy) for xy in zip(df['long'], df['lat'])]
gdf = GeoDataFrame(df, geometry=geometry)   

world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
gdf.plot(ax=world.plot(figsize=(10, 6)), marker='o', color='red', markersize=5);

## Data Cleaning

### Dropping columns/instances

In [ ]:
#drop columns that don't add any value to our analysis

df.drop(['Unnamed: 0', 'id', 'url', 'region_url', 'lat', 'long', 'VIN', 'image_url',
         'description', 'geometry', 'posting_date'], axis = 1, inplace = True)
df.head()

In [ ]:
df.shape

In [ ]:
#keep only 'clean' cars & missing values
#missing, parts only, etc. cars don't fit the goal of our analysis

df = df[((df['title_status'] != 'missing') & (df['title_status'] != 'parts only') 
    & (df['title_status'] != 'salvage') & (df['title_status'] != 'rebuilt')
   & (df['title_status'] != 'lien'))]
df.shape

In [ ]:
#delete unrealistically over-/underpriced cars (over $500k or under $100)
#sometimes prices are $0 or $123456789 because the owner wants to agree on the price in person

df = df[((df['price'] < 500000) & (df['price'] > 100))]
df.shape

In [ ]:
starting_year = 1960
x = (df.groupby(['year'],as_index = False).count())['year'].tolist()
y = (df.groupby(['year'],as_index = False).count())['price'].tolist()
fig = plt.figure(figsize=(10,5))
plt.bar(x, y, width=1)
plt.axvline(starting_year, color = 'red')

In [ ]:
#delete cars built before 1960 (outliers)

df = df[df['year'] > 1960]
df.shape

In [ ]:
#delete top 1% of odometer values (outliers)

top1 = np.nanpercentile(df['odometer'], 99)
print('the top 1% mileage is', top1, 'miles')
df = df[df['odometer'] < top1]
df.shape

In [ ]:
#size is perfect multicollinear with car model => drop size column

df.drop('size', axis = 1, inplace = True)
df.shape

In [ ]:
#delete unknown fuel, transmission and title_status instances

df.dropna(subset = ['fuel', 'transmission', 'title_status'], axis = 0, inplace = True)
df.shape

In [ ]:
#delete all instances with unknown manufacturer

df = df[~df['manufacturer'].isnull()]
df.shape

In [ ]:
#delete all instances with unknown model

df = df[~df['model'].isnull()]
df.shape

### Replacing missing values with placeholders 

In [ ]:
#replace missing condition values with 'not specified'

df['condition'].fillna('not specified', inplace = True)
df['condition'].unique()

In [ ]:
#replace missing color with 'unknown'

df['paint_color'].fillna('unknown', inplace = True)
df['paint_color'].unique()

### Cleaning car model values and reducing the number of categories

In [ ]:
#first word is most important for 'model': many car model categories refer to the same model but are written differently 
#(i.e. 'f150 good condition' and 'F - 150 four wheel drive' refer to the same model: f150)
#=> keep only first word and remove spaces, special characters, uppercase letters, etc.

#delete instances with a model that appears less than 150 times => reduce amount of categories by omitting uncommon types

model_list = df['model'].tolist()
model_list = map(str, model_list)
model_list = [x.lower().strip() for x in model_list]
my_list = [car_model.split()[0] for car_model in model_list]
my_list = [x.replace(' ', '').replace('-', '').replace('/', '') for x in my_list]

df['car_model'] = my_list
df['car_model'] = df['manufacturer'] + " " + df['car_model']

new_car_models = (df.groupby(['car_model'], as_index= False).count())[['manufacturer', 'car_model', 'price']]
new_car_models = new_car_models.rename(columns={'price': 'count'})
only_common_models = new_car_models[new_car_models['count'] > 150]

print('total car models: ', len(new_car_models['count']))
print('remaining car models: ', len(only_common_models['count']))
print('decreasing ', round((1-len(only_common_models['count'])/len(new_car_models['count'])) * 100, 2),'% of the number of car models')
print()
print('total instances: ', len(df['price']))
print('remaining instances: ', sum(only_common_models['count']))
print('removing ', round((1-sum(only_common_models['count'])/len(df['price'])) * 100, 2),'% of the instances')

In [ ]:
#150 seems like the reasonable choice w.r.t. trade-off reducing categories vs keeping enough data

removed_models_list = []
removed_instances_list = []

for i in range(0,1000):
    only_common_models_v2 = new_car_models[new_car_models['count']>i]
    models_removed = 1-len(only_common_models_v2['count'])/len(new_car_models['count'])
    instances_removed = 1-sum(only_common_models_v2['count'])/len(df['price'])
    
    removed_models_list.append(models_removed)
    removed_instances_list.append(instances_removed)

fig = plt.figure(figsize=(7,7))
plt.plot(removed_models_list, label = 'Percentage of unqiue models removed')
plt.plot(removed_instances_list, label = 'Percentage of instances removed')
plt.axvline(150 , color = 'red', label = 'Cut-off')
plt.ylabel('Percentage Remaining')
plt.xlabel('Car model frequency cut-off')
plt.legend()

In [ ]:
#committing car model cleaning to original dataframe

df = df[df['car_model'].isin(only_common_models['car_model'].tolist())]
df.drop('model', axis = 1, inplace = True)
df.shape

In [ ]:
df.head()

### Replacing missing values with reasonable proxies

In [ ]:
#replace missing number of cylinders with median of car model & delete 'other'

df = df[df['cylinders'] != 'other']
dftest = df.copy()
test = df[df['cylinders'].notnull()].copy()
test['cylinders'] = [int(cyl.split()[0]) for cyl in test['cylinders']]
med = test.groupby('car_model')['cylinders'].median()
merged = pd.merge(dftest, med, on = 'car_model', how = 'left')
merged['cylinders_y'].fillna(med.median(), inplace = True)
merged['cylinders_x'].fillna(merged['cylinders_y'], inplace = True)

li = []
for cyl in merged['cylinders_x']:
    if type(cyl) == str:
        li.append(cyl)
    else:
        li.append(str(int(cyl)) + " cylinders")

df['cylinders'] = li
df['cylinders'].unique()

In [ ]:
#replace drive with mode drive of car model

dftest = df.copy()
test = df[df['drive'].notnull()].copy()
mode = test.groupby('car_model')['drive'].agg(pd.Series.mode)
merged = pd.merge(dftest, mode, on = 'car_model', how = 'left')
merged['drive_x'].fillna(merged['drive_y'], inplace = True)

df['drive'] = merged['drive_x'].tolist()
df['drive'].unique()

In [ ]:
#replace type with mode type of car model

dftest = df.copy()
test = df[df['type'].notnull()].copy()
mode = test.groupby('car_model')['type'].agg(pd.Series.mode)
for i in mode.index:
    if type(mode[i]) != str:
        mode[i] = mode[i][0]
merged = pd.merge(dftest, mode, on = 'car_model', how = 'left')
merged['type_x'].fillna(merged['type_y'], inplace = True)

df['type'] = merged['type_x'].tolist()
df['type'].unique()

In [ ]:
#all the missing values are handled

df.isnull().sum()

### Handling price outliers

In [ ]:
#get rid of overpriced cars: overpriced = more than 3 times the average for this model

df = df[df['price'] < (3 * df.groupby('car_model')['price'].transform('mean'))]
df.shape

### Dropping columns with redundant information

In [ ]:
#get rid of 'title_status' since it is 'clean' for every instance

df.drop('title_status', axis = 1, inplace = True)
df.shape

In [ ]:
#region & state contain same information (certain region is always in the same state)

df.drop('region', axis = 1, inplace = True)
df.shape

### Making a backup and saving the data

In [ ]:
clean = df.copy()

In [ ]:
df.to_csv('clean.csv', index = False)

## Feature Engineering/Selection

### Creating modeling and k-fold cross validation functions

In [ ]:
#creates model as defined in 'reg' and returns test & train evaluation metrics
#default is 80/20 train-test split: industry standard

def updateModel(datafr, test_percentage = 0.2, seed = 7):
    
    #train-test split
    X = datafr.drop('price', axis = 1)
    y = datafr['price']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = test_percentage, random_state = seed)
    X_train = pd.get_dummies(X_train, drop_first = True)
    X_test = pd.get_dummies(X_test, drop_first = True)
    
    #feature scaling
    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)
    
    #making model + predicting
    reg.fit(X_train, y_train)
    y_pred_test = reg.predict(X_test)
    y_pred_train = reg.predict(X_train)
    
    #computing test & train metrics
    test_metrics = {'r2': round(r2_score(y_test, y_pred_test) * 100, 2),
           'mae': round(mean_absolute_error(y_test, y_pred_test), 2),
           'mse': round(mean_squared_error(y_test, y_pred_test), 2),
           'mape': round(mean_absolute_percentage_error(y_test, y_pred_test) * 100, 2)
           }
    train_metrics = {'r2': round(r2_score(y_train, y_pred_train) * 100, 2),
           'mae': round(mean_absolute_error(y_train, y_pred_train), 2),
           'mse': round(mean_squared_error(y_train, y_pred_train), 2),
           'mape': round(mean_absolute_percentage_error(y_train, y_pred_train) * 100, 2)
           }
    
    return {'test_metrics': test_metrics,
            'train_metrics': train_metrics}

In [ ]:
#creates model as defined in 'reg' and returns k fold cross validation metrics
#default is 5-fold CV: industry standard & default scoring is MAE because of easy interpretation

def get_kCVscores(datafr, k = 5, scoring = 'neg_mean_absolute_error'):
    X = datafr.drop('price', axis = 1)
    X = pd.get_dummies(X, drop_first = True)
    y = datafr['price']
    return (cross_val_score(reg, X, y, cv = k, scoring = scoring) * (-1))

In [ ]:
#K-means clustering function
def k_means_clustering(datafr, k):
    df_to_return = datafr.copy()
    df_with_dummies = pd.get_dummies(datafr, drop_first = True)
    km = KMeans(n_clusters = k)
    df_to_return['cluster'] = km.fit_predict(df_with_dummies)
    
    return df_to_return

### Baseline Linear Regression before feature engineering

In [ ]:
reg = LinearRegression()

In [ ]:
#initial linear regression model

init_metrics = updateModel(df)
def print_metrics(metrics):
    print('test performance:', metrics['test_metrics'])
    print('initial test performance:', init_metrics['test_metrics'])
    print()
    print('train performance:', metrics['train_metrics'])
    print('initial train performance:', init_metrics['train_metrics'])
    print()
    print('change in test MAE:', round((init_metrics['test_metrics']['mae'] - metrics['test_metrics']['mae']) / 
      (init_metrics['test_metrics']['mae']) * 100, 2), '%')
print_metrics(init_metrics)

Reasonably poor performance but no sign of overfitting

### Grouping states into North-East, Mid-West, South and West

In [ ]:
regions_dict = {"state": ["al","ak","az","ar","ca","co","ct","de","dc","fl","ga","hi","id","il","in","ia","ks","ky","la","me","md","ma","mi","mn","ms","mo","mt","ne","nv","nh","nj","nm","ny","nc","nd","oh","ok","or","pa","ri","sc","sd","tn","tx","ut","vt","va","wa","wv","wi","wy"], 
         "region": ["south","west","west","south","west","west","north-east","south","south","south","south","west","west","mid-west","mid-west","mid-west","mid-west","south","south","north-east","south","north-east","mid-west","mid-west","south","mid-west","west","mid-west","west","north-east","north-east","west","north-east","south","mid-west","mid-west","south","west","north-east","north-east","south","mid-west","south","south","west","north-east","south","west","south","mid-west","west"]}
dfState = pd.merge(df, pd.DataFrame(regions_dict), on = 'state', how = 'left').drop('state', axis = 1)
dfState

In [ ]:
#new metrics

binned_states_metrics = updateModel(dfState)
print_metrics(binned_states_metrics)

almost no difference => let's try just removing region & state altogether

### Dropping 'state' variable

In [ ]:
dfDropState = df.drop('state', axis = 1)
dfDropState

In [ ]:
#new metrics

drop_state_metrics = updateModel(dfDropState)
print_metrics(drop_state_metrics)

test MAE became (marginally) worse, so let's keep the 'state' feature

### Excluding car_model (many categories)

In [ ]:
dfNoCarModel = df.drop('car_model', axis = 1)
dfNoCarModel

In [ ]:
no_car_model_metrics = updateModel(dfNoCarModel)
print_metrics(no_car_model_metrics)

even though 'car_model' has many categories, it still clearly provides useful information

### Binning 'year' variable

In [ ]:
dfBinnedYear = df.copy()
dfBinnedYear['year'] = pd.cut(df['year'], 3, labels = ['vintage', 'medium age', 'recent'])
dfBinnedYear

In [ ]:
binned_year_metrics = updateModel(dfBinnedYear)
print_metrics(binned_year_metrics)

performance became significantly worse

### Creating feature 'age'

In [ ]:
dfAge = df.copy()
dfAge['age'] = [2021 - x for x in df['year'].tolist()]
dfAge.drop('year', axis=1, inplace = True)
dfAge

In [ ]:
age_metrics = updateModel(dfAge)
print_metrics(age_metrics)

performance became marginally worse => let's not include 'age'

### Creating categories for 'age'

In [ ]:
#we can clearly see a U-shaped pricing behavior in function of car age
#in addition, we can identify age categories with similar pricing behavior

dfAgeCat = dfAge.copy()
thresholds = [50,40,30,15,14,13,12,11,10,9,8,7,6,5,4,3,2,1]
groups = ['over 50','40-49','30-39','15-29','14','13','12','11','10','9','8','7','6','5','4','3','2','1']

x = (dfAgeCat.groupby(['age'],as_index = False).mean())['age'].tolist()
y = (dfAgeCat.groupby(['age'],as_index = False).mean())['price'].tolist()
fig = plt.figure(figsize=[10,5])
plt.ylabel('Mean price')
plt.xlabel('Age (years)')
plt.bar(x, y, width=1)
for t in thresholds:
    plt.axvline(t-0.5, color = 'red')

plt.axvline(-0.5, color = 'red')
plt.axvline(60.5, color = 'red')

In [ ]:
arr = np.array(dfAgeCat['age'].tolist())
for t in thresholds:
    prevArr = arr
    arr = np.where(prevArr >= t, -t, prevArr)
    
prevArr = arr
dfAgeCat['age_group'] = np.where(prevArr >= 0, 'new', prevArr).tolist()

for i in range(len(thresholds)):
     toReplace = str(float(-thresholds[i]))
     dfAgeCat['age_group'].replace(toReplace, groups[i], inplace =  True)

dfAgeCat.drop('age', axis=1, inplace=True)    
print(dfAgeCat['age_group'].unique())
dfAgeCat

In [ ]:
age_cat_metrics = updateModel(dfAgeCat)
print_metrics(age_cat_metrics)

despite the fact that including 'age' decreased performance, including age categories clearly improved performance without any sign of overfitting

### Price vs features analysis

In [ ]:
dfPlots = dfAge.copy()
for feat in dfAgeCat.drop(['price','odometer','car_model','age_group'], axis = 1).columns:
    print(feat)
    x = (dfPlots.groupby(feat, as_index = False).mean()).sort_values(by = ['price'])[feat]
    y = (dfPlots.groupby(feat, as_index = False).mean()).sort_values(by = ['price'])['price']
    fig = plt.figure(figsize=[10,5])
    plt.xlabel(feat)
    plt.ylabel('Mean price')
    plt.xticks(rotation = 90)
    plt.bar(x, y, width=1)
    plt.show()

### Condition as a numerical feature

In [ ]:
dfCondition = df.copy()

dfCondition['condition'].replace('new', 6, inplace =  True)
dfCondition['condition'].replace('like new', 5, inplace =  True)
dfCondition['condition'].replace('excellent', 4, inplace =  True)
dfCondition['condition'].replace('good', 3, inplace =  True)
dfCondition['condition'].replace('fair', 2, inplace =  True)
dfCondition['condition'].replace('salvage', 1, inplace =  True)
dfCondition['condition'].replace('not specified', 3, inplace =  True)

dfCondition['condition'].unique()

In [ ]:
condition_metrics = updateModel(dfCondition)
print_metrics(condition_metrics)

no significant effect on performance

### Color categories

In [ ]:
dfColors = df.copy()
dfColors['paint_color'].replace(['unknown', 'blue', 'silver', 'grey', 'green', 'custom', 'yellow', 'brown', 'purple'], 'other_colors', inplace =  True)
dfColors['paint_color'].unique()

In [ ]:
color_metrics = updateModel(dfColors)
print_metrics(color_metrics)

no significant effect on performance

### Miles per year

In [ ]:
dfMPY = dfAge.copy()
MPY_temp = []

dfMPY['miles_per_year'] = [dfMPY['odometer'][i]/(dfMPY['age'][i]+0.001) for i in dfMPY.index.tolist()]
dfMPY.drop(['odometer', 'age'], axis=1, inplace=True)
dfMPY

In [ ]:
MPY_metrics = updateModel(dfMPY)
print_metrics(MPY_metrics)

significant negative impact on performance

### Excluding number of cylinders

In [ ]:
dfNoCyl = df.drop('cylinders', axis = 1)
dfNoCyl

In [ ]:
no_cyl_metrics = updateModel(dfNoCyl)
print_metrics(no_cyl_metrics)

marginal negative impact on performance

### Excluding 'type'

In [ ]:
dfNoType = df.drop('type', axis = 1)
dfNoType

In [ ]:
no_type_metrics = updateModel(dfNoType)
print_metrics(no_type_metrics)

no significant effect on performance

### Committing feature engineering changes to df

In [ ]:
#only creating age_groups had a positive impact on performance
#dropping 'type' and binning 'state' into regions had no real impact on performance but made the model more interpretable

df = dfAgeCat
df = pd.merge(df, pd.DataFrame(regions_dict), on = 'state', how = 'left').drop('state', axis = 1)
df.drop('type', axis = 1, inplace = True)
df

## Model Creation

### Linear Regression

In [ ]:
reg = LinearRegression()

In [ ]:
#linear regression model on the feature engineered df

initLR = updateModel(df)
print_metrics(initLR)

Feature engineering had a clear positive impact on performance without a sign of overfitting

### Feature Importance

In [ ]:
cols = pd.get_dummies(df, drop_first = True).columns

In [ ]:
importance = reg.coef_

temp = pd.DataFrame({'feat n°': [x for x in range(len(importance))],'importance': importance})
temp = temp.sort_values('importance')
worst10 = temp[:10]
best10 = temp[-10:]
best10['feat'] = cols[best10['feat n°']]
worst10['feat'] = cols[worst10['feat n°']]

In [ ]:
plt.xticks(rotation = 90)
plt.bar(best10['feat'], best10['importance'])

In [ ]:
plt.xticks(rotation = 90)
plt.bar(worst10['feat'], worst10['importance'])

### Random Forest Regressor

In [ ]:
#20-tree RF

reg = RandomForestRegressor(n_estimators = 20, random_state = 7)

In [ ]:
initRF = updateModel(df)
print_metrics(initRF)

In [ ]:
kcv = get_kCVscores(df)
kcv

In [ ]:
print('Cross validated MAE mean:', kcv.mean())
print('Cross validated MAE standard deviation:', kcv.std())

In [ ]:
#100-tree RF

reg = RandomForestRegressor(n_estimators = 100, random_state = 7)

In [ ]:
init100RF = updateModel(df)
print_metrics(init100RF)

the 20-tree random forest drastically outperforms the linear regression

however, cross validation clearly signals an overfitting problem

### Boosted Tree Regressor

In [ ]:
reg = GradientBoostingRegressor(n_estimators = 50, learning_rate = 0.1, max_depth = 1, random_state = 7, loss = 'ls')

In [ ]:
initBoost = updateModel(df)
print_metrics(initBoost)

Boosted Tree performs significantly worse than linear regression

### Neural Network

In [ ]:
reg = MLPRegressor()

In [ ]:
initNN = updateModel(df)
print_metrics(initNN)

Better than LR but not better than RF

### Clustering

In [ ]:
#Elbow plot values
distortions = []
df_with_dummies = pd.get_dummies(df.copy(), drop_first = True)

K = range(1,10)
for k in K:
    km = KMeans(n_clusters=k)
    km.fit(df_with_dummies)
    distortions.append(km.inertia_)

#Plotting
plt.figure(figsize=(16,8))
plt.plot(K, distortions, 'bx-')
plt.xlabel('k')
plt.ylabel('Distortion')
plt.title('Elbow Method to determine the optimal k value')
plt.show()

In [ ]:
df_clustering_k2 = k_means_clustering(df, 2)
df_clustering_k2

In [ ]:
df_clustering_k3 = k_means_clustering(df, 3)
df_clustering_k3

In [ ]:
pd.get_dummies(df_clustering_k2, drop_first = True).groupby(['cluster']).median()

interpretation: cluster 0 are more expensive cars with low mileage while cluster 1 are cheap cars with a lot of mileage

In [ ]:
pd.get_dummies(df_clustering_k3, drop_first = True).groupby(['cluster']).median()

interpretation: analogous but with an extra 'medium priced' category

In [ ]:
reg = LinearRegression()

In [ ]:
df_to_use = pd.get_dummies(df_clustering_k2, drop_first = True)
k = 2
print('For k = ', k)

list_of_results = []
for i in range(k):
    cluster_df = (df_to_use[df_to_use['cluster']==i]).copy()
    res = updateModel(cluster_df)
    print_metrics(res)
    list_of_results.append(res)

clust_k2_LR_results = list_of_results

In [ ]:
df_to_use = pd.get_dummies(df_clustering_k3, drop_first = True)
k = 3
print('For k = ', k)

list_of_results = []
for i in range(k):
    cluster_df = (df_to_use[df_to_use['cluster']==i]).copy()
    res = updateModel(cluster_df)
    print_metrics(res)
    list_of_results.append(res)

clust_k3_LR_results = list_of_results

K=2 seems to improve performance the most and the clusters are also more interpretable

In [ ]:
reg = RandomForestRegressor(n_estimators = 20, random_state = 7)

In [ ]:
df_to_use = pd.get_dummies(df_clustering_k2, drop_first = True)
k = 2
print('For k = ', k)

list_of_results = []
for i in range(k):
    cluster_df = (df_to_use[df_to_use['cluster']==i]).copy()
    res = updateModel(cluster_df)
    print_metrics(res)
    list_of_results.append(res)

clust_k2_RF_results = list_of_results

In [ ]:
print_metrics(initRF)

### Final Model

In [ ]:
dfFinal = df_clustering_k2.copy()
dfFinal

In [ ]:
finalPerformance = clust_k2_RF_results
print('Cluster 1:')
print_metrics(finalPerformance[0])
print()
print('Cluster 2:')
print_metrics(finalPerformance[1])

In [ ]:
finalModel = reg
reg

In [ ]:
finalClusters = df_clustering_k2.copy()
pd.get_dummies(finalClusters, drop_first = True).groupby(['cluster']).median()

In [ ]:
dfFinal.to_csv('final')